<center><h1>NoSQL Database Labs: BigTable and Neo4J</h1></center>

<h2>Overview</h2>

<ul>
    <li>What is BigTable?</li>
    <li>BigTable Storage Model</li>
    <li>Cloud Execution</li>
    <li>Neo4J</li>
    <li>Neo4J on the Cloud</li>
    <li>Cypher Scripts</li>
    <li>Neo4J in Python</li>
    <li>Readings</li>
</ul>

<h1>Reminder: SQL vs. NoSQL Performance at SELECT Queries</h1>

<center><figure><img src="https://www.andrew.cmu.edu/user/mfarag/static/sql_no_sql_performance.png"/><figcaption>Database Comparison from Profil website, accessible from <a href="https://profil-software.com/blog/development/database-comparison-sql-vs-nosql-mysql-vs-postgresql-vs-redis-vs-mongodb/">here</a></figcaption></figure></center>

<h1>MongoDB does not have the best performance!</h1>

<center><figure><img src="https://www.andrew.cmu.edu/user/mfarag/static/no_sql_performance.jpg"/><figcaption>NoSQL DB Comparison, featured in the details of this <a href="https://www.techniajz.com/blog-detail/comparison-between-databases-hbase-cassandra-and-mongodb">article</a></figcaption></figure></center>

<h1>BigTable</h1>

<ul>
    <li>Bigtable is engineered to handle structured data at a vast scale, managing petabytes across thousands of servers for high-demand applications</li>
    <li>Bigtable supports diverse workloads from high-throughput batch processing to real-time, latency-sensitive data serving. This includes 60+ Google products such as Google Analytics and Google Earth</li>
    <li>Bigtable clusters vary in size, from a few to thousands of servers, efficiently managing hundreds of terabytes of data to meet the unique needs of each Google service.</li>
</ul>

<h2>BigTable Performance Statistics</h2>

<center><figure><img src="https://www.andrew.cmu.edu/user/mfarag/static/bigtable_performance.jpg"/><figcaption>BigTable Performance on 8 vCPU Machine, accessible from <a href="https://cloudplatform.googleblog.com/2015/05/introducing-Google-Cloud-Bigtable.html">here</a></figcaption></figure></center>

<h3>Important Notes</h3>
<ul>
    <li>Bigtable is suitable for tables with sparse data.</li> 
    <li>if a column is not used in a particular row, it does not take up any space.</li>
</ul>


<h2>BigTable Storage Model</h2>

<ul>
    <li>Bigtable stores data in massively scalable tables</li> 
    <li>Each table is a sorted key/value map.</li>
    <li>The table is composed of rows, each of which typically describes a single entity, and columns, which contain individual values for each row.</li>
    <li>Each row is indexed by a single row key, and columns that are related to one another are typically grouped into a column family.</li>
    <li>Each column is identified by a combination of the column family and a column qualifier, which is a unique name within the column family.</li>
    </ul>

<ul>
<li>Each row/column intersection can contain multiple cells.</li>
    <li>Each cell contains a unique timestamped version of the data for that row and column.</li>
        <li>Storing multiple cells in a column provides a record of how the stored data for that row and column have changed over time.</li>
    </ul>

<center><img src="https://www.andrew.cmu.edu/user/mfarag/static/bigtable-model.png"/></center>

<h2>BigTable Performance Statistics</h2>

<center><figure><img src="https://www.andrew.cmu.edu/user/mfarag/static/bigtable_performance.jpg"/><figcaption>BigTable Performance on 8 vCPU Machine, accessible from <a href="https://cloudplatform.googleblog.com/2015/05/introducing-Google-Cloud-Bigtable.html">here</a></figcaption></figure></center>

<h2>Run BigTable Locally via Docker</h2>

You can run a BigTable emulator Locally. To do so, you need to pull Google Cloud SDK Container and run it. More information can be found <a href="https://hub.docker.com/r/google/cloud-sdk">here</a>

<ul>
    <li>Set the <b>BIGTABLE_EMULATOR_HOST</b> environment variable to <b>localhost:8086</b>. More information can be found on <a href="https://cloud.google.com/bigtable/docs/emulator">https://cloud.google.com/bigtable/docs/emulator</a></li>
    <li>Run the docker pull command for the image: <b>docker pull google/cloud-sdk</b></li>
    <li>Execute the docker run command: <b>docker run -p 127.0.0.1:8086:8086 --rm -ti google/cloud-sdk gcloud beta emulators bigtable start --host-port=0.0.0.0:8086</b></li>
    <li>In your terminal, run <b>docker container ls</b> or check your docker desktop to see if the container is running</li>
</ul>

<h3>Install Required Packages</h3>

In [2]:
!pip install google-cloud-bigtable
!pip install google-cloud-happybase

<h3>Initialize the Application</h3>

In [3]:
from google.cloud import bigtable
from google.cloud import happybase
from google.cloud.bigtable import column_family

#Populate project_id and instance_id if you are running on the cloud
# project_id = "forward-byte-434113-d3"
instance_id = "toolchains"

# client = bigtable.Client(project=project_id, admin=True)
client = bigtable.Client( admin=True)
instance = client.instance(instance_id)


KeyboardInterrupt: 

<h3>Create Tables</h3>

In [4]:
table_id = 'test'
print("Creating the {} table.".format(table_id))
table = instance.table(table_id)

print("Creating column family cf1 with Max Version GC rule...")
# Create a column family with GC policy : most recent N versions
# Define the GC policy to retain only the most recent 2 versions
max_versions_rule = column_family.MaxVersionsGCRule(2)
column_family_id = "cf1"
column_families = {column_family_id: max_versions_rule}
if not table.exists():
    table.create(column_families=column_families)
else:
    print("Table {} already exists.".format(table_id))


Creating the test table.
Creating column family cf1 with Max Version GC rule...


RetryError: Timeout of 60.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8086: Failed to connect to remote host: connect: Connection refused (61)

<h3>Insert Rows into Tables</h3>

In [16]:
import datetime

print("Writing some greetings to the table.")
greetings = ["Hello World!", "Hello Cloud Bigtable!", "Hello Python!"]
rows = []
column = "greeting".encode()
for i, value in enumerate(greetings):
    # Note: This example uses sequential numeric IDs for simplicity,
    # but this can result in poor performance in a production
    # application.  Since rows are stored in sorted order by key,
    # sequential keys can result in poor distribution of operations
    # across nodes.
    #
    # For more information about how to design a Bigtable schema for
    # the best performance, see the documentation:
    #
    #     https://cloud.google.com/bigtable/docs/schema-design
    row_key = "greeting{}".format(i).encode()
    row = table.direct_row(row_key)
    row.set_cell(
        column_family_id, column, value, timestamp=datetime.datetime.utcnow()
    )
    rows.append(row)
table.mutate_rows(rows)

Writing some greetings to the table.


[, , ]

<h3>Retrieve All Rows in BigTable Table!</h3>

In [17]:
print("Scanning for all greetings:")
partial_rows = table.read_rows()

for row in partial_rows:
    cell = row.cells[column_family_id][column][0]
    print(cell.value.decode("utf-8"))

Scanning for all greetings:
Hello World!
Hello Cloud Bigtable!
Hello Python!


<h3>Delete Tables</h3>

In [18]:
print("Deleting the {} table.".format(table_id))
table.delete()

Deleting the test table.


<h2>Take Home Excercise</h2>

<h3>Read the Example for More Inforamtion <a href="https://cloud.google.com/bigtable/docs/samples-python-hello">https://cloud.google.com/bigtable/docs/samples-python-hello</a></h3>

<h2>BigTable Cloud Execution</h2>

To create Google BigTable instance on GCP, conduct the following activities:
<ul>
<li>1. Download and Configure Google Cloud SDK from this URL: <a href="https://cloud.google.com/sdk/docs/install">https://cloud.google.com/sdk/docs/install</a></li>
<li>2. Enable the Billing on Your GCP Account</li>
<li>3. Ensure you have a project created</li>
<li>4. Enable the BigTable API and Create a BigTable Instance from <a href="https://console.cloud.google.com/bigtable/instances">https://console.cloud.google.com/bigtable/instances</a></li>
 <li>5. Update your project ID and Instance ID in the code.</li>
     </ul>

<h3>If authorization is not configured on your local machine, generate Key JSON file for Your Service Account from GCP and Add it at the same folder next to your Jupyter Notebook. Delete your BIGTABLE_EMULATOR_HOST variable and restart your Jupyter terminal before running the following snippet</h3>

In [3]:
import os
#Use the correct filename
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./cmu-class.json"

<h1>Graph Databases</h1>

<h2>Why Graph Databases/Stores?!</h2>

<ul>
<li>Graph databases/Stores naturally represent entities and relationships, reducing <b>impedance mismatch</b> compared to relational databases.</li>
<li>Graph DBs  align closely with how applications and humans view data, making it easier to model real-world domains.</li>
<li>No need for complex joins to reconstruct relationships, improving performance and simplicity.</li>
</ul>

<h2>Graph DB Use Cases</h2>

<ul>
<li>Cybercrime Networks: Graph databases map relationships in cybercrime activity, supporting investigators in analyzing attack patterns and identifying threats.</li>
<li>Recommendation Engines: Graph databases excel in constructing recommendation systems by modeling intricate relationships between users, products, and preferences. This facilitates the discovery of complex patterns, such as "customers who purchased X also tend to purchase Y," enabling more refined and personalized recommendations.</li>
<li>Fraud Detection: By mapping and analyzing the complex web of relationships between transactions, accounts, and entities, graph databases can detect sophisticated fraud patterns.</li>
</ul>

<h1>Neo4J</h1>

<ul>
<li>Founded in 2000.</li>
<li>Available in open-source and commercial editions.</li>
<li>Highly scalable.</li>
<li>Uses a powerful query lanaguage named Cypher which aids in traversing graphs smoothly.</li>
</ul>

<h2>Neo4J Performance Statistics</h2>

<center><figure><img src="https://www.andrew.cmu.edu/user/mfarag/static/neo4j_oracle_performance_comparison.png"/><figcaption>Oracle vs. Neo4J Query Performance for Finding all of a plant's ancestors at Monsanto, accessible from <a href="https://neo4j.com/blog/oracle-rdbms-neo4j-fully-sync-data/">here</a></figcaption></figure></center>

<h2>Neo4J on the Cloud</h2>
We will use AuraDB which offers Free Tier option

<ul>
    <li>Register for a new Neo4J account via this URL: <a href="https://neo4j.com/product/auradb/">https://neo4j.com/product/auradb/</a></li>
<li>Verify Your Email. Upon verifying your email (on the same tab of email verification), navigate to the console page, accept the privacy policy, and sign up for a free tier.</li>
<li>Download the Instance Password as a text file to your machine.</li>
<li>Open the Instance.</li>
<li>Click Connect (and save the connection URI).</li>
</ul>

<h2>Cypher</h2>

<ul>
<li>Cypher streamlines querying by focusing on what to retrieve or create, abstracting the how for seamless interaction with the graph.</li>
<li>Cypher uses intuitive symbols—parentheses for nodes and arrows for relationships—offering a clear, visual representation of graph structures</li>
    <li>Check Cypher documentation <a href="https://neo4j.com/docs/cypher-manual/current/introduction/">here</a></li>
</ul>

<h3>Example: Track Inter-relationships between Select Persons</h3>

<center><figure><img src="https://www.andrew.cmu.edu/user/mfarag/static/graph_example.png"/><figcaption>Graph diagram for the target entities and inter-relationships</figcaption></figure></center>

<h3>Create a new Graph</h3>

In [ ]:
CREATE (einstein:Person {name: "Albert Einstein", birthYear: 1879, famousFor: "Physics"}),
       (bohr:Person {name: "Niels Bohr", birthYear: 1885, famousFor: "QuantumMechanics"}),
       (heisenberg:Person {name: "Werner Heisenberg", birthYear: 1901}),
       (shchrodinger:Person {name: "Erwin Schrodinger", birthYear: 1887}),
       (curie:Person {name: "Marie Curie", birthYear: 1867, famousFor: "Radiation"}),

       (einstein)-[:COLLABORATED_WITH {domain:"QuantumMechanics"}]-> (bohr),
       (bohr)-[:MENTORED]-> (heisenberg),
       (heisenberg)-[:COLLABORATED_WITH]-> (shchrodinger),
       (shchrodinger)-[:AWARDED_LIKE]-> (curie);

<h3>Display All Nodes in a Graph</h3>

In [ ]:
MATCH (n)
RETURN n;

<h3>Filter by the Existence of an Attribute</h3>
Display Famous Persons

In [ ]:
MATCH (n:Person)
WHERE (n.famousFor IS NOT NULL)
RETURN n;

<h3>Retrieve an Attribute for a Matching Node</h3>
Find The Birth Year for Albert Einstein

In [ ]:
MATCH (n:Person {name: "Albert Einstein"})
RETURN n.birthYear;

<h3>Find the Nodes with a Specific Relationship</h3>
Find all the mentors in our Graph

In [ ]:
MATCH (n:Person) -[:MENTORED]-> (m:Person)
Return n

<h3>Filter Nodes Based on Relationship Attributes</h3>
Find all the collaborators in the Quantum Mechanics Domain

In [ ]:
MATCH (n:Person) -[collaboration:COLLABORATED_WITH]-> (m:Person)
WHERE collaboration.domain = "QuantumMechanics"
Return n,m

<h3>Insert a new Relationship in a Graph</h3>
Add New Relationships between Curie and Einstein

In [ ]:
MATCH (n1:Person {name: "Marie Curie"}), (n2:Person {name: "Albert Einstein"})
CREATE (n1)-[:AWARDED_LIKE]->(n2);

<h3>Delete all the Nodes in a Graph</h3>

In [ ]:
MATCH (n)
DETACH DELETE n;

<h2>Neo4J in Python</h2>
Install Dependencies

In [5]:
!pip install neo4j

In [5]:
from neo4j import GraphDatabase

class Neo4JConnection:
    def __init__(self, uri, user, password):
        """Initialize the Neo4j connection with URI, username, and password."""
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        """Close the Neo4j connection."""
        if self.driver:
            self.driver.close()

    def execute_query(self, query, parameters=None):
        """Execute a Cypher query and return the result."""
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result.data()

In [6]:
# Function to create nodes and relationships as per Query 1
def create_nodes_and_relationships(connection):
    query = """
    CREATE (einstein:Person {name: "Albert Einstein", birthYear: 1879, famousFor: "Physics"}),
           (bohr:Person {name: "Niels Bohr", birthYear: 1885, famousFor: "QuantumMechanics"}),
           (heisenberg:Person {name: "Werner Heisenberg", birthYear: 1901}),
           (shchrodinger:Person {name: "Erwin Schrodinger", birthYear: 1887}),
           (curie:Person {name: "Marie Curie", birthYear: 1867, famousFor: "Radiation"}),
           
           (einstein)-[:COLLABORATED_WITH {domain:"QuantumMechanics"}]-> (bohr),
           (bohr)-[:MENTORED]-> (heisenberg),
           (heisenberg)-[:COLLABORATED_WITH]-> (shchrodinger),
           (shchrodinger)-[:AWARDED_LIKE]-> (curie);
    """
    connection.execute_query(query)
    print("Nodes and relationships created.")




In [7]:
# Function for adding a new relationship between Marie Curie and Albert Einstein
def add_awarded_like_relationship(connection):
    query = """
    MATCH (n1:Person {name: "Marie Curie"}), (n2:Person {name: "Albert Einstein"})
    CREATE (n1)-[:AWARDED_LIKE]->(n2);
    """
    connection.execute_query(query)
    print("AWARDED_LIKE relationship created between Marie Curie and Albert Einstein.")


In [1]:
# Function for returning people who mentored others
def get_mentors(connection):
    query = """
    MATCH (n:Person) -[:MENTORED]-> (m:Person)
    RETURN n;
    """
    result = connection.execute_query(query)
    for record in result:
        print(record)



In [9]:
# Function for deleting all nodes
def delete_all_nodes(connection):
    query = """
    MATCH (n)
    DETACH DELETE n;
    """
    connection.execute_query(query)
    print("All nodes deleted.")

In [10]:
def main():
    # Replace with your actual Neo4j AuraDB credentials
    uri = "neo4j+s://a3349088.databases.neo4j.io"
    user = "neo4j"
    password = "DNmGymR3u9A9Z95wu0-739aoziP8Qqb5i-r-7apmAM4"

    # Initialize Neo4j connection
    neo4j_conn = Neo4JConnection(uri, user, password)

    try:
        # Execute the queries
        delete_all_nodes(neo4j_conn)
        create_nodes_and_relationships(neo4j_conn)
        add_awarded_like_relationship(neo4j_conn)
        get_mentors(neo4j_conn)
        delete_all_nodes(neo4j_conn)
    finally:
        # Close the connection when done
        neo4j_conn.close()


if __name__ == "__main__":
    main()

All nodes deleted.
Nodes and relationships created.
AWARDED_LIKE relationship created between Marie Curie and Albert Einstein.
{'n': {'famousFor': 'QuantumMechanics', 'birthYear': 1885, 'name': 'Niels Bohr'}}
All nodes deleted.


<h2>Readings</h2>

<ul>
<li>Usefule Tools for Neo4J: <a href="https://neo4j.com/docs/tools/">https://neo4j.com/docs/tools/</a></li>
    <li>More on Creating Nodes from <a href="https://www.tutorialspoint.com/neo4j/neo4j_cql_creating_nodes.htm">here</a></li>
</ul>